In [1]:
import pandas as pd
from pathlib import Path
import pandas_datareader.data as reader
import datetime as dt
from MCForecastTools import MCSimulation

# For visualizations:
import seaborn as sns

# PART I: DATA CAPTURE AND ORGANIZATION

In [2]:
end = dt.date(2022, 12, 31)
start = dt.date(2020, 1, 1)

ticker1 = ["BTC-USD"]
ticker2 = ["VOT"]  #iShares Russell Mid-Cap GROWTH ETF, a midcap growth fund. Chose midcap to factor out size.
ticker3 = ["IWS"]  #iShares Russell Mid-Cap VALUE ETF, a midcap value fund. Chose midcap to factor out size.
ticker4 = ["MGC"]   #Vanguard Mega Cap ETF, a fund with LARGE CAPS only to feature size (large cap end of the spectrum).
ticker5 = ["EWSC"]  #Invesco S&P SmallCap 600 Equal Weight ETF       iShares Micro-Cap ETF, a fund with SMALL CAPS only to feature size (small cap end of the spectrum).
ticker6 = ["SPY"]

In [3]:
# Get BTC closing prices

btc_prices = reader.get_data_yahoo(ticker1, start, end)
display(btc_prices.head())
btc_prices.tail()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2020-01-01,7200.174316,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
2020-01-02,6985.470215,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2020-01-03,7344.884277,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
2020-01-04,7410.656738,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
2020-01-05,7411.317383,7411.317383,7544.497070,7400.535645,7410.451660,19725074095


Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2022-05-08,34059.265625,34059.265625,35502.941406,33878.964844,35502.941406,36763041910
2022-05-09,30296.953125,30296.953125,34222.074219,30296.953125,34060.015625,63355494961
2022-05-10,31022.906250,31022.906250,32596.308594,29944.802734,30273.654297,59811038817
2022-05-11,28936.355469,28936.355469,32013.402344,28170.414062,31016.183594,70388855818
2022-05-12,28436.689453,28436.689453,30016.650391,26350.490234,29061.509766,87832674304


In [4]:
btc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
btc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-01,7200.174316,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
2020-01-02,6985.470215,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2020-01-03,7344.884277,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
2020-01-04,7410.656738,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
2020-01-05,7411.317383,7411.317383,7544.497070,7400.535645,7410.451660,19725074095


In [5]:
btc_prices = btc_prices[["open", "high", "low", "close", "volume", "adj close"]]
btc_prices.drop(columns="adj close", axis=1, inplace=True)
btc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095


In [6]:
btc_only_prices = pd.concat([btc_prices], axis=1, keys=['BTC USD'])
btc_only_prices

BTC USD                                            \
                    open          high           low         close   
Date                                                                 
2020-01-01   7194.892090   7254.330566   7174.944336   7200.174316   
2020-01-02   7202.551270   7212.155273   6935.270020   6985.470215   
2020-01-03   6984.428711   7413.715332   6914.996094   7344.884277   
2020-01-04   7345.375488   7427.385742   7309.514160   7410.656738   
2020-01-05   7410.451660   7544.497070   7400.535645   7411.317383   
...                  ...           ...           ...           ...   
2022-05-08  35502.941406  35502.941406  33878.964844  34059.265625   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   
2022-05-12  29061.509766  30016.650391  26350.490234  28436.689453   

                         
                 volume  
Date                     
2020-01-01  18565664997  
2020-01-02  20802083465  
2020-01-03  28111481032  
2020-01-04  18444271275  
2020-01-05  19725074095  
...                 ...  
2022-05-08  36763041910  
2022-05-09  63355494961  
2022-05-10  59811038817  
2022-05-11  70388855818  
2022-05-12  87832674304  

[863 rows x 5 columns]

In [7]:
iwp_prices = reader.get_data_yahoo(ticker2, start, end)
display(iwp_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,VOT,VOT,VOT,VOT,VOT,VOT
Date,,,,,,
2020-01-02,158.127548,160.039993,160.039993,158.880005,159.770004,259200.0
2020-01-03,157.870667,159.779999,159.960007,157.789993,158.199997,134400.0
2020-01-06,158.315292,160.229996,160.270004,158.550003,158.669998,148600.0
2020-01-07,158.423996,160.339996,160.710007,159.800003,160.149994,99100.0
2020-01-08,158.967407,160.889999,161.660004,160.369995,160.369995,182900.0


In [8]:
iwp_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwp_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,158.127548,160.039993,160.039993,158.880005,159.770004,259200.0
2020-01-03,157.870667,159.779999,159.960007,157.789993,158.199997,134400.0
2020-01-06,158.315292,160.229996,160.270004,158.550003,158.669998,148600.0
2020-01-07,158.423996,160.339996,160.710007,159.800003,160.149994,99100.0
2020-01-08,158.967407,160.889999,161.660004,160.369995,160.369995,182900.0


In [9]:
iwp_prices = iwp_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwp_prices.drop(columns="adj close", axis=1, inplace=True)
iwp_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,159.770004,160.039993,158.880005,160.039993,259200.0
2020-01-03,158.199997,159.960007,157.789993,159.779999,134400.0
2020-01-06,158.669998,160.270004,158.550003,160.229996,148600.0
2020-01-07,160.149994,160.710007,159.800003,160.339996,99100.0
2020-01-08,160.369995,161.660004,160.369995,160.889999,182900.0


In [10]:
iwp_only_prices = pd.concat([iwp_prices], axis=1, keys=['IWP'])
iwp_only_prices

IWP                                              
                  open        high         low       close    volume
Date                                                                
2020-01-02  159.770004  160.039993  158.880005  160.039993  259200.0
2020-01-03  158.199997  159.960007  157.789993  159.779999  134400.0
2020-01-06  158.669998  160.270004  158.550003  160.229996  148600.0
2020-01-07  160.149994  160.710007  159.800003  160.339996   99100.0
2020-01-08  160.369995  161.660004  160.369995  160.889999  182900.0
...                ...         ...         ...         ...       ...
2022-05-05  202.539993  202.830002  193.539993  195.710007  290100.0
2022-05-06  194.130005  194.130005  187.929993  190.869995  493200.0
2022-05-09  187.350006  188.119995  179.500000  180.419998  479500.0
2022-05-10  184.350006  185.380005  177.000000  181.250000  538400.0
2022-05-11  180.229996  183.919998  176.350006  176.649994  511000.0

[595 rows x 5 columns]

In [11]:
iws_prices = reader.get_data_yahoo(ticker3, start, end)
display(iws_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWS,IWS,IWS,IWS,IWS,IWS
Date,,,,,,
2020-01-02,90.591820,94.570000,95.209999,94.019997,95.160004,557000.0
2020-01-03,90.266129,94.230003,94.389999,93.650002,93.879997,443700.0
2020-01-06,90.294853,94.260002,94.269997,93.680000,93.830002,664600.0
2020-01-07,90.122429,94.080002,94.279999,93.839996,94.099998,358100.0
2020-01-08,90.160751,94.120003,94.480003,93.889999,94.169998,386400.0


In [12]:
iws_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iws_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,90.591820,94.570000,95.209999,94.019997,95.160004,557000.0
2020-01-03,90.266129,94.230003,94.389999,93.650002,93.879997,443700.0
2020-01-06,90.294853,94.260002,94.269997,93.680000,93.830002,664600.0
2020-01-07,90.122429,94.080002,94.279999,93.839996,94.099998,358100.0
2020-01-08,90.160751,94.120003,94.480003,93.889999,94.169998,386400.0


In [13]:
iws_prices = iws_prices[["open", "high", "low", "close", "volume", "adj close"]]
iws_prices.drop(columns="adj close", axis=1, inplace=True)
iws_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,95.160004,95.209999,94.019997,94.570000,557000.0
2020-01-03,93.879997,94.389999,93.650002,94.230003,443700.0
2020-01-06,93.830002,94.269997,93.680000,94.260002,664600.0
2020-01-07,94.099998,94.279999,93.839996,94.080002,358100.0
2020-01-08,94.169998,94.480003,93.889999,94.120003,386400.0


In [14]:
iws_only_prices = pd.concat([iws_prices], axis=1, keys=['IWS'])
iws_only_prices

IWS                                               
                  open        high         low       close     volume
Date                                                                 
2020-01-02   95.160004   95.209999   94.019997   94.570000   557000.0
2020-01-03   93.879997   94.389999   93.650002   94.230003   443700.0
2020-01-06   93.830002   94.269997   93.680000   94.260002   664600.0
2020-01-07   94.099998   94.279999   93.839996   94.080002   358100.0
2020-01-08   94.169998   94.480003   93.889999   94.120003   386400.0
...                ...         ...         ...         ...        ...
2022-05-05  116.000000  116.279999  112.519997  113.519997   527400.0
2022-05-06  113.040001  113.370003  111.099998  112.620003  1243800.0
2022-05-09  111.320000  111.519997  108.639999  109.000000   907000.0
2022-05-10  110.309998  110.610001  107.160004  108.709999   934800.0
2022-05-11  108.769997  110.809998  107.430000  107.540001  1586100.0

[595 rows x 5 columns]

In [15]:
mgc_prices = reader.get_data_yahoo(ticker4, start, end)
display(mgc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,MGC,MGC,MGC,MGC,MGC,MGC
Date,,,,,,
2020-01-02,109.172379,112.889999,112.889999,112.110001,112.379997,135600.0
2020-01-03,108.379379,112.070000,112.489998,111.529999,111.750000,69200.0
2020-01-06,108.833908,112.540001,112.550003,111.199997,111.379997,192200.0
2020-01-07,108.524437,112.220001,112.449997,112.099998,112.410004,75000.0
2020-01-08,109.143356,112.860001,113.300003,112.180000,112.180000,113600.0


In [16]:
mgc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
mgc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,109.172379,112.889999,112.889999,112.110001,112.379997,135600.0
2020-01-03,108.379379,112.070000,112.489998,111.529999,111.750000,69200.0
2020-01-06,108.833908,112.540001,112.550003,111.199997,111.379997,192200.0
2020-01-07,108.524437,112.220001,112.449997,112.099998,112.410004,75000.0
2020-01-08,109.143356,112.860001,113.300003,112.180000,112.180000,113600.0


In [17]:
mgc_prices = mgc_prices[["open", "high", "low", "close", "volume", "adj close"]]
mgc_prices.drop(columns="adj close", axis=1, inplace=True)
mgc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,112.379997,112.889999,112.110001,112.889999,135600.0
2020-01-03,111.750000,112.489998,111.529999,112.070000,69200.0
2020-01-06,111.379997,112.550003,111.199997,112.540001,192200.0
2020-01-07,112.410004,112.449997,112.099998,112.220001,75000.0
2020-01-08,112.180000,113.300003,112.180000,112.860001,113600.0


In [18]:
mgc_only_prices = pd.concat([mgc_prices], axis=1, keys=['MGC'])
mgc_only_prices

MGC                                              
                  open        high         low       close    volume
Date                                                                
2020-01-02  112.379997  112.889999  112.110001  112.889999  135600.0
2020-01-03  111.750000  112.489998  111.529999  112.070000   69200.0
2020-01-06  111.379997  112.550003  111.199997  112.540001  192200.0
2020-01-07  112.410004  112.449997  112.099998  112.220001   75000.0
2020-01-08  112.180000  113.300003  112.180000  112.860001  113600.0
...                ...         ...         ...         ...       ...
2022-05-05  148.360001  148.429993  143.000000  144.419998  243100.0
2022-05-06  143.600006  144.910004  141.610001  143.690002  237200.0
2022-05-09  141.369995  141.669998  138.369995  138.960007  285400.0
2022-05-10  141.240005  141.699997  137.990005  139.339996  313200.0
2022-05-11  138.929993  140.889999  136.729996  137.000000  271300.0

[595 rows x 5 columns]

In [19]:
iwc_prices = reader.get_data_yahoo(ticker5, start, end)
display(iwc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,EWSC,EWSC,EWSC,EWSC,EWSC,EWSC
Date,,,,,,
2020-01-02,55.771477,56.820000,57.040001,56.410000,57.040001,1600.0
2020-01-03,55.781296,56.830002,56.830002,56.529999,56.529999,1700.0
2020-01-06,55.938339,56.990002,56.990002,56.540001,56.560001,1900.0
2020-01-07,55.565350,56.610001,56.709999,56.430000,56.709999,4300.0
2020-01-08,55.477016,56.520000,56.570000,56.450001,56.500000,600.0


In [20]:
iwc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,55.771477,56.820000,57.040001,56.410000,57.040001,1600.0
2020-01-03,55.781296,56.830002,56.830002,56.529999,56.529999,1700.0
2020-01-06,55.938339,56.990002,56.990002,56.540001,56.560001,1900.0
2020-01-07,55.565350,56.610001,56.709999,56.430000,56.709999,4300.0
2020-01-08,55.477016,56.520000,56.570000,56.450001,56.500000,600.0


In [21]:
iwc_prices = iwc_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwc_prices.drop(columns="adj close", axis=1, inplace=True)
iwc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,57.040001,57.040001,56.410000,56.820000,1600.0
2020-01-03,56.529999,56.830002,56.529999,56.830002,1700.0
2020-01-06,56.560001,56.990002,56.540001,56.990002,1900.0
2020-01-07,56.709999,56.709999,56.430000,56.610001,4300.0
2020-01-08,56.500000,56.570000,56.450001,56.520000,600.0


In [22]:
iwc_only_prices = pd.concat([iwc_prices], axis=1, keys=['IWC'])
iwc_only_prices

IWC                                         
                 open       high        low      close  volume
Date                                                          
2020-01-02  57.040001  57.040001  56.410000  56.820000  1600.0
2020-01-03  56.529999  56.830002  56.529999  56.830002  1700.0
2020-01-06  56.560001  56.990002  56.540001  56.990002  1900.0
2020-01-07  56.709999  56.709999  56.430000  56.610001  4300.0
2020-01-08  56.500000  56.570000  56.450001  56.520000   600.0
...               ...        ...        ...        ...     ...
2022-05-05  75.980003  75.980003  73.599998  73.849998  2500.0
2022-05-06  73.379997  73.379997  72.680000  72.940002  1100.0
2022-05-09  71.910004  71.910004  70.629997  70.949997  4200.0
2022-05-10  71.570000  71.570000  69.599998  70.480003  5700.0
2022-05-11  70.690002  71.730003  69.230003  69.279999  5900.0

[595 rows x 5 columns]

In [23]:
spy_prices = reader.get_data_yahoo(ticker6, start, end)
display(spy_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2020-01-02,313.744446,324.869995,324.890015,322.529999,323.540009,59151200.0
2020-01-03,311.368774,322.410004,323.640015,321.100006,321.160004,77709700.0
2020-01-06,312.556641,323.640015,323.730011,320.359985,320.489990,55653900.0
2020-01-07,311.677826,322.730011,323.540009,322.239990,323.019989,40496400.0
2020-01-08,313.338898,324.450012,325.779999,322.670013,322.940002,68296000.0


In [24]:
spy_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
spy_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,313.744446,324.869995,324.890015,322.529999,323.540009,59151200.0
2020-01-03,311.368774,322.410004,323.640015,321.100006,321.160004,77709700.0
2020-01-06,312.556641,323.640015,323.730011,320.359985,320.489990,55653900.0
2020-01-07,311.677826,322.730011,323.540009,322.239990,323.019989,40496400.0
2020-01-08,313.338898,324.450012,325.779999,322.670013,322.940002,68296000.0


In [25]:
spy_prices = spy_prices[["open", "high", "low", "close", "volume", "adj close"]]
spy_prices.drop(columns="adj close", axis=1, inplace=True)
spy_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,323.540009,324.890015,322.529999,324.869995,59151200.0
2020-01-03,321.160004,323.640015,321.100006,322.410004,77709700.0
2020-01-06,320.489990,323.730011,320.359985,323.640015,55653900.0
2020-01-07,323.019989,323.540009,322.239990,322.730011,40496400.0
2020-01-08,322.940002,325.779999,322.670013,324.450012,68296000.0


In [26]:
spy_only_prices = pd.concat([spy_prices], axis=1, keys=['SPY'])
spy_only_prices

SPY                                                 
                  open        high         low       close       volume
Date                                                                   
2020-01-02  323.540009  324.890015  322.529999  324.869995   59151200.0
2020-01-03  321.160004  323.640015  321.100006  322.410004   77709700.0
2020-01-06  320.489990  323.730011  320.359985  323.640015   55653900.0
2020-01-07  323.019989  323.540009  322.239990  322.730011   40496400.0
2020-01-08  322.940002  325.779999  322.670013  324.450012   68296000.0
...                ...         ...         ...         ...          ...
2022-05-05  424.549988  425.000000  409.440002  413.809998  172929100.0
2022-05-06  411.100006  414.799988  405.730011  411.339996  151671300.0
2022-05-09  405.100006  406.410004  396.500000  398.170013  155586100.0
2022-05-10  404.489990  406.079987  394.820007  399.089996  132497200.0
2022-05-11  398.070007  404.040009  391.959991  392.750000  142089600.0

[595 rows x 5 columns]

In [27]:
# Concatenate the ticker DataFrames
prices_btc_iwp = pd.concat([btc_prices, iwp_prices], axis=1, keys=["BTC-USD", "IWP"])
prices_btc_iws = pd.concat([btc_prices, iws_prices], axis=1, keys=["BTC-USD", "IWS"])
prices_btc_mgc = pd.concat([btc_prices, mgc_prices], axis=1, keys=["BTC-USD", "MGC"])
prices_btc_iwc = pd.concat([btc_prices, iwc_prices], axis=1, keys=["BTC-USD", "IWC"])
prices_btc_spy = pd.concat([btc_prices, spy_prices], axis=1, keys=["BTC-USD", "SPY"])

prices_btc_iwp = prices_btc_iwp.dropna()
prices_btc_iws = prices_btc_iws.dropna()
prices_btc_mgc = prices_btc_mgc.dropna()
prices_btc_iwc = prices_btc_iwc.dropna()
prices_btc_spy = prices_btc_spy.dropna()

display(prices_btc_iwp.tail())
display(prices_btc_iws.tail())
display(prices_btc_mgc.tail())
display(prices_btc_iwc.tail())
prices_btc_spy.tail()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   

                                IWP                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-05  43106256317  202.539993  202.830002  193.539993  195.710007   
2022-05-06  37795577489  194.130005  194.130005  187.929993  190.869995   
2022-05-09  63355494961  187.350006  188.119995  179.500000  180.419998   
2022-05-10  59811038817  184.350006  185.380005  177.000000  181.250000   
2022-05-11  70388855818  180.229996  183.919998  176.350006  176.649994   

                      
              volume  
Date                  
2022-05-05  290100.0  
2022-05-06  493200.0  
2022-05-09  479500.0  
2022-05-10  538400.0  
2022-05-11  511000.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   

                                IWS                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-05  43106256317  116.000000  116.279999  112.519997  113.519997   
2022-05-06  37795577489  113.040001  113.370003  111.099998  112.620003   
2022-05-09  63355494961  111.320000  111.519997  108.639999  109.000000   
2022-05-10  59811038817  110.309998  110.610001  107.160004  108.709999   
2022-05-11  70388855818  108.769997  110.809998  107.430000  107.540001   

                       
               volume  
Date                   
2022-05-05   527400.0  
2022-05-06  1243800.0  
2022-05-09   907000.0  
2022-05-10   934800.0  
2022-05-11  1586100.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   

                                MGC                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-05  43106256317  148.360001  148.429993  143.000000  144.419998   
2022-05-06  37795577489  143.600006  144.910004  141.610001  143.690002   
2022-05-09  63355494961  141.369995  141.669998  138.369995  138.960007   
2022-05-10  59811038817  141.240005  141.699997  137.990005  139.339996   
2022-05-11  70388855818  138.929993  140.889999  136.729996  137.000000   

                      
              volume  
Date                  
2022-05-05  243100.0  
2022-05-06  237200.0  
2022-05-09  285400.0  
2022-05-10  313200.0  
2022-05-11  271300.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   

                               IWC                                           
                 volume       open       high        low      close  volume  
Date                                                                         
2022-05-05  43106256317  75.980003  75.980003  73.599998  73.849998  2500.0  
2022-05-06  37795577489  73.379997  73.379997  72.680000  72.940002  1100.0  
2022-05-09  63355494961  71.910004  71.910004  70.629997  70.949997  4200.0  
2022-05-10  59811038817  71.570000  71.570000  69.599998  70.480003  5700.0  
2022-05-11  70388855818  70.690002  71.730003  69.230003  69.279999  5900.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  31016.183594  32013.402344  28170.414062  28936.355469   

                                SPY                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-05  43106256317  424.549988  425.000000  409.440002  413.809998   
2022-05-06  37795577489  411.100006  414.799988  405.730011  411.339996   
2022-05-09  63355494961  405.100006  406.410004  396.500000  398.170013   
2022-05-10  59811038817  404.489990  406.079987  394.820007  399.089996   
2022-05-11  70388855818  398.070007  404.040009  391.959991  392.750000   

                         
                 volume  
Date                     
2022-05-05  172929100.0  
2022-05-06  151671300.0  
2022-05-09  155586100.0  
2022-05-10  132497200.0  
2022-05-11  142089600.0

# PART II: MONTE CARLOS 

In [28]:
#MC_btc = MCSimulation(
 #   portfolio_data = btc_only_prices,
  #  weights = [1],
   # num_simulation = 500,
    #num_trading_days = 126)

# Review the simulation input data
#MC_btc.portfolio_data.head()

In [29]:
MC_iwp = MCSimulation(
    portfolio_data = iwp_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iwp.portfolio_data.head()

IWP                                                \
                  open        high         low       close    volume   
Date                                                                   
2020-01-02  159.770004  160.039993  158.880005  160.039993  259200.0   
2020-01-03  158.199997  159.960007  157.789993  159.779999  134400.0   
2020-01-06  158.669998  160.270004  158.550003  160.229996  148600.0   
2020-01-07  160.149994  160.710007  159.800003  160.339996   99100.0   
2020-01-08  160.369995  161.660004  160.369995  160.889999  182900.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.001625  
2020-01-06     0.002816  
2020-01-07     0.000687  
2020-01-08     0.003430

In [30]:
MC_iws = MCSimulation(
    portfolio_data = iws_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iws.portfolio_data.head()

IWS                                                        
                 open       high        low      close    volume daily_return
Date                                                                         
2020-01-02  95.160004  95.209999  94.019997  94.570000  557000.0          NaN
2020-01-03  93.879997  94.389999  93.650002  94.230003  443700.0    -0.003595
2020-01-06  93.830002  94.269997  93.680000  94.260002  664600.0     0.000318
2020-01-07  94.099998  94.279999  93.839996  94.080002  358100.0    -0.001910
2020-01-08  94.169998  94.480003  93.889999  94.120003  386400.0     0.000425

In [31]:
MC_mgc = MCSimulation(
    portfolio_data = mgc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_mgc.portfolio_data.head()

MGC                                                \
                  open        high         low       close    volume   
Date                                                                   
2020-01-02  112.379997  112.889999  112.110001  112.889999  135600.0   
2020-01-03  111.750000  112.489998  111.529999  112.070000   69200.0   
2020-01-06  111.379997  112.550003  111.199997  112.540001  192200.0   
2020-01-07  112.410004  112.449997  112.099998  112.220001   75000.0   
2020-01-08  112.180000  113.300003  112.180000  112.860001  113600.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.007264  
2020-01-06     0.004194  
2020-01-07    -0.002843  
2020-01-08     0.005703

In [32]:
MC_iwc = MCSimulation(
    portfolio_data = iwc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iwc.portfolio_data.head()

IWC                                                      
                 open       high        low      close  volume daily_return
Date                                                                       
2020-01-02  57.040001  57.040001  56.410000  56.820000  1600.0          NaN
2020-01-03  56.529999  56.830002  56.529999  56.830002  1700.0     0.000176
2020-01-06  56.560001  56.990002  56.540001  56.990002  1900.0     0.002815
2020-01-07  56.709999  56.709999  56.430000  56.610001  4300.0    -0.006668
2020-01-08  56.500000  56.570000  56.450001  56.520000   600.0    -0.001590

In [33]:
MC_spy = MCSimulation(
    portfolio_data = spy_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_spy.portfolio_data.head()

SPY                                                  \
                  open        high         low       close      volume   
Date                                                                     
2020-01-02  323.540009  324.890015  322.529999  324.869995  59151200.0   
2020-01-03  321.160004  323.640015  321.100006  322.410004  77709700.0   
2020-01-06  320.489990  323.730011  320.359985  323.640015  55653900.0   
2020-01-07  323.019989  323.540009  322.239990  322.730011  40496400.0   
2020-01-08  322.940002  325.779999  322.670013  324.450012  68296000.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.007572  
2020-01-06     0.003815  
2020-01-07    -0.002812  
2020-01-08     0.005330

In [34]:
MC_btc_iwp = MCSimulation(
    portfolio_data = prices_btc_iwp,
    weights = [.5, .5],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_iwp.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                                IWP                                      \
           daily_return        open        high         low       close   
Date                                                                      
2020-01-02          NaN  159.770004  160.039993  158.880005  160.039993   
2020-01-03     0.051452  158.199997  159.960007  157.789993  159.779999   
2020-01-06     0.057773  158.669998  160.270004  158.550003  160.229996   
2020-01-07     0.050774  160.149994  160.710007  159.800003  160.339996   
2020-01-08    -0.010269  160.369995  161.660004  160.369995  160.889999   

                                   
              volume daily_return  
Date                               
2020-01-02  259200.0          NaN  
2020-01-03  134400.0    -0.001625  
2020-01-06  148600.0     0.002816  
2020-01-07   99100.0     0.000687  
2020-01-08  182900.0     0.003430

In [35]:
MC_btc_iws = MCSimulation(
    portfolio_data = prices_btc_iws,
    weights = [.5, .5],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_iws.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                               IWS                                             \
           daily_return       open       high        low      close    volume   
Date                                                                            
2020-01-02          NaN  95.160004  95.209999  94.019997  94.570000  557000.0   
2020-01-03     0.051452  93.879997  94.389999  93.650002  94.230003  443700.0   
2020-01-06     0.057773  93.830002  94.269997  93.680000  94.260002  664600.0   
2020-01-07     0.050774  94.099998  94.279999  93.839996  94.080002  358100.0   
2020-01-08    -0.010269  94.169998  94.480003  93.889999  94.120003  386400.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.003595  
2020-01-06     0.000318  
2020-01-07    -0.001910  
2020-01-08     0.000425

In [36]:
MC_btc_mgc = MCSimulation(
    portfolio_data = prices_btc_mgc,
    weights = [.5, .5],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_mgc.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                                MGC                                      \
           daily_return        open        high         low       close   
Date                                                                      
2020-01-02          NaN  112.379997  112.889999  112.110001  112.889999   
2020-01-03     0.051452  111.750000  112.489998  111.529999  112.070000   
2020-01-06     0.057773  111.379997  112.550003  111.199997  112.540001   
2020-01-07     0.050774  112.410004  112.449997  112.099998  112.220001   
2020-01-08    -0.010269  112.180000  113.300003  112.180000  112.860001   

                                   
              volume daily_return  
Date                               
2020-01-02  135600.0          NaN  
2020-01-03   69200.0    -0.007264  
2020-01-06  192200.0     0.004194  
2020-01-07   75000.0    -0.002843  
2020-01-08  113600.0     0.005703

In [37]:
MC_btc_iwc = MCSimulation(
    portfolio_data = prices_btc_iwc,
    weights = [.5, .5],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_iwc.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                               IWC                                           \
           daily_return       open       high        low      close  volume   
Date                                                                          
2020-01-02          NaN  57.040001  57.040001  56.410000  56.820000  1600.0   
2020-01-03     0.051452  56.529999  56.830002  56.529999  56.830002  1700.0   
2020-01-06     0.057773  56.560001  56.990002  56.540001  56.990002  1900.0   
2020-01-07     0.050774  56.709999  56.709999  56.430000  56.610001  4300.0   
2020-01-08    -0.010269  56.500000  56.570000  56.450001  56.520000   600.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03     0.000176  
2020-01-06     0.002815  
2020-01-07    -0.006668  
2020-01-08    -0.001590

In [38]:
MC_btc_spy = MCSimulation(
    portfolio_data = prices_btc_spy,
    weights = [.5, .5],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_spy.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                                SPY                                      \
           daily_return        open        high         low       close   
Date                                                                      
2020-01-02          NaN  323.540009  324.890015  322.529999  324.869995   
2020-01-03     0.051452  321.160004  323.640015  321.100006  322.410004   
2020-01-06     0.057773  320.489990  323.730011  320.359985  323.640015   
2020-01-07     0.050774  323.019989  323.540009  322.239990  322.730011   
2020-01-08    -0.010269  322.940002  325.779999  322.670013  324.450012   

                                     
                volume daily_return  
Date                                 
2020-01-02  59151200.0          NaN  
2020-01-03  77709700.0    -0.007572  
2020-01-06  55653900.0     0.003815  
2020-01-07  40496400.0    -0.002812  
2020-01-08  68296000.0     0.005330

In [39]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
#MC_btc.calc_cumulative_return()

In [40]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.997107,0.970016,0.977312,1.018359,1.018011,0.993736,1.015154,1.020672,0.992674,1.021014,...,1.010537,0.978592,1.000106,0.981999,0.976755,1.029416,0.995954,0.975889,0.995652,1.022074
2,0.986579,0.999189,0.937930,0.994056,0.993761,0.987354,0.981393,1.025376,0.971776,1.021322,...,1.003898,0.934811,0.996487,0.980095,0.984281,1.014256,0.987357,0.989330,0.980183,1.014099
3,0.988096,1.005781,0.925922,0.989959,0.986075,0.991995,1.028666,1.030402,0.996593,1.020934,...,1.010670,0.950066,0.998142,0.958476,0.960273,1.016938,0.958384,1.006412,1.001784,1.021641
4,0.997376,1.024553,0.886134,1.031379,0.997457,1.006371,1.011424,1.005653,1.000756,1.022287,...,1.009993,0.961027,0.966588,0.947765,0.944662,0.998621,0.938617,1.005481,0.984015,1.039323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.235924,0.989529,1.219432,1.434533,0.904668,0.837122,1.342559,0.962261,0.974638,1.259449,...,0.914959,0.999459,1.017145,1.489610,0.924413,1.775126,1.087895,1.451890,0.870243,1.600952
249,1.234048,0.979499,1.239113,1.425575,0.899086,0.827976,1.330985,0.964016,0.974071,1.237062,...,0.921983,0.994099,1.010386,1.460224,0.912680,1.766528,1.083926,1.497640,0.884310,1.604831
250,1.261970,0.980625,1.233316,1.405051,0.885686,0.826734,1.315521,0.966504,0.966091,1.269208,...,0.930284,0.979558,0.992994,1.482124,0.911577,1.827370,1.123536,1.499852,0.885948,1.625348
251,1.271407,0.953785,1.268456,1.416525,0.905147,0.814081,1.299395,0.966640,0.963279,1.287036,...,0.929730,0.963561,1.021119,1.521519,0.918955,1.837897,1.105183,1.530332,0.839754,1.613056


In [41]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.030397,0.989207,0.999887,0.963358,1.000260,1.029286,1.008406,0.985692,1.024680,1.027526,...,1.013670,0.986177,1.004373,0.986710,0.989324,1.001182,0.992714,0.997860,0.998090,0.980047
2,1.039977,1.011455,1.001717,0.983033,0.997389,1.007495,0.993990,0.998152,1.020264,1.014289,...,1.001385,0.996533,0.990574,0.997948,0.977735,0.989546,0.961569,0.988791,0.994045,1.007893
3,1.066807,0.993873,0.999968,0.952636,0.992238,0.998689,1.007720,0.958488,0.988797,1.040503,...,1.004974,0.996974,0.982102,0.994120,0.979836,0.988497,0.939090,0.961980,0.985488,1.023684
4,1.076716,1.015600,0.980575,0.987633,0.969398,0.997263,1.013450,0.939726,0.984902,1.056404,...,1.017349,0.948149,0.975130,0.981450,0.985998,0.972678,0.971766,0.985575,0.975819,1.010127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.212251,0.955878,0.748759,2.796555,0.738035,1.372841,0.894926,0.644418,0.987428,0.806311,...,0.831905,1.574465,0.595219,0.976415,0.473312,1.596470,1.491611,0.540642,1.489669,0.625391
249,1.223470,0.947965,0.723599,2.840791,0.756709,1.378832,0.896375,0.667808,0.973685,0.826142,...,0.827504,1.542549,0.603516,0.960890,0.470337,1.555347,1.474983,0.555419,1.515082,0.620475
250,1.169743,0.932979,0.709552,2.889385,0.761375,1.401425,0.906126,0.672484,0.974794,0.816186,...,0.811820,1.590911,0.600998,1.006701,0.459189,1.544467,1.515731,0.547485,1.495777,0.591075
251,1.129804,0.950256,0.703645,2.855963,0.767267,1.402654,0.943393,0.675573,0.977318,0.813714,...,0.807928,1.566077,0.601631,1.018856,0.462592,1.559896,1.502387,0.531679,1.495676,0.596046


In [42]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.

/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()



Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mo

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.003281,0.988314,0.989943,0.997771,1.013107,0.987922,0.979366,0.995269,1.007720,1.013629,...,1.010272,0.976214,1.045210,0.986529,1.002072,1.008968,1.000869,0.989440,0.993732,0.992984
2,1.001955,0.988477,1.011161,0.991727,1.020992,0.974498,0.953273,0.975906,1.001884,1.017869,...,0.986048,0.953240,1.051169,0.992185,0.982983,0.997059,0.975989,0.974087,0.974289,0.977804
3,0.978839,0.976726,1.002734,0.987637,1.046579,0.983568,0.994223,0.938056,1.009933,1.008793,...,0.983280,0.948573,1.058605,1.005976,1.025925,0.989057,0.964343,0.965971,0.980246,0.978437
4,0.977359,0.992054,1.005487,1.010683,1.042762,0.941148,0.996558,0.946733,1.011199,1.003648,...,0.994490,0.940148,1.056334,1.002092,1.048317,1.008335,0.959337,0.983508,0.962316,1.007788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.109660,1.243076,0.996391,1.194250,1.173308,1.229008,0.923647,1.146969,0.753254,1.254561,...,1.136532,1.053563,1.664806,1.707578,1.306539,0.830065,1.321075,1.157449,1.033225,1.068237
249,1.160143,1.232327,1.015214,1.200935,1.169004,1.206761,0.948390,1.162803,0.740256,1.258595,...,1.179073,1.043841,1.651284,1.704144,1.292890,0.816650,1.324732,1.184796,1.046561,1.090645
250,1.141661,1.241967,1.042805,1.177348,1.159252,1.226600,0.956725,1.168665,0.759580,1.286007,...,1.157172,1.033702,1.633629,1.691098,1.277724,0.825867,1.315864,1.190167,1.051015,1.109293
251,1.148350,1.246520,1.028416,1.216662,1.167667,1.248929,0.954142,1.183903,0.737551,1.269392,...,1.175662,1.020759,1.698375,1.731792,1.270883,0.821189,1.301281,1.221576,1.077610,1.108876


In [43]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.965950,1.006530,1.013418,0.976388,0.985563,1.005910,0.971268,0.990227,0.991867,1.005152,...,0.994854,1.007070,1.003263,0.998731,1.010429,1.025989,0.991950,1.011013,0.986524,0.978159
2,0.949542,1.012247,1.003186,0.972700,0.989407,1.009836,0.940444,0.983249,1.013352,1.010178,...,0.983211,1.010408,0.971518,1.015990,1.020822,0.998901,1.003276,0.993671,1.007084,0.973236
3,0.985523,1.011316,0.965600,0.953810,0.999836,1.009890,0.961185,0.991347,1.012618,0.999231,...,0.982271,1.006313,0.932066,0.983827,1.029309,0.996803,0.977475,1.022167,1.008010,0.967713
4,0.991293,1.053403,0.925258,0.978916,1.002957,1.026182,0.972886,1.021930,1.015795,1.021576,...,1.007500,0.988158,0.906150,0.985863,1.026107,1.000761,0.965065,1.035578,1.043268,0.945004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.257135,1.041345,0.869012,1.221188,0.929532,1.179054,1.972763,0.875787,0.756152,1.420611,...,1.009910,1.217277,1.102511,0.656666,1.123161,0.842076,1.209542,1.525905,1.503371,1.331821
249,1.223862,1.047292,0.877100,1.230055,0.898820,1.161591,1.980192,0.874560,0.761993,1.398173,...,1.021012,1.186882,1.098668,0.650631,1.110253,0.805530,1.220771,1.525240,1.446146,1.317001
250,1.197099,1.056304,0.874237,1.244667,0.934695,1.200341,2.084491,0.892899,0.753986,1.466110,...,0.987572,1.200894,1.108817,0.660686,1.134829,0.821993,1.166092,1.564749,1.476777,1.281731
251,1.204981,1.000545,0.892017,1.229288,0.922400,1.192962,2.097550,0.923379,0.742785,1.499410,...,0.972092,1.145660,1.115705,0.659612,1.152836,0.819850,1.115450,1.572326,1.437277,1.303945


In [44]:
MC_spy.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.965456,1.008788,1.008537,0.993951,1.020389,1.001097,1.028952,1.005590,0.980038,0.965820,...,0.980363,1.004051,1.037972,1.003711,1.009173,0.990896,0.986711,1.003407,1.000827,0.974393
2,0.968848,1.006273,0.983970,1.008043,1.006638,1.008751,1.049991,1.007282,0.975777,0.985579,...,0.978796,0.997062,1.027532,0.979755,1.006502,0.976094,0.965262,1.005532,1.013170,0.984781
3,0.976330,1.008638,0.996395,0.995916,0.987055,1.001967,1.013996,1.017128,0.979060,0.986205,...,0.987632,1.009742,1.032392,0.969577,1.008512,0.984969,0.984985,1.031692,1.011651,0.993160
4,0.989457,0.995453,1.038299,0.981593,1.001059,1.018897,1.011922,1.052355,0.967515,0.977304,...,1.002610,1.028782,1.060184,0.981857,1.001789,1.010165,0.988283,1.036422,0.991485,0.995040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.816400,0.963284,0.975099,1.348956,0.932564,1.256605,1.144458,1.312847,1.241564,1.241121,...,0.951342,1.273864,1.029143,0.921062,0.839735,1.120249,1.163589,0.962362,1.020362,1.177524
249,0.811741,0.973688,0.954805,1.348237,0.920190,1.279203,1.147759,1.308518,1.216416,1.255350,...,0.975125,1.273967,1.014234,0.934190,0.825878,1.104091,1.153949,0.954106,1.057452,1.151641
250,0.813948,0.973430,0.944200,1.304817,0.920178,1.315779,1.118937,1.321295,1.214316,1.258273,...,0.967000,1.250582,1.020171,0.950473,0.818757,1.129246,1.147854,0.949182,1.087122,1.160763
251,0.828299,0.967668,0.973789,1.292233,0.904679,1.350072,1.111424,1.316400,1.214547,1.280545,...,0.969147,1.269921,1.057152,0.948319,0.797568,1.123023,1.125427,0.952990,1.114157,1.172140


In [45]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_btc_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.

/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()



Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Mo

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.984838,1.026210,1.017163,1.005415,1.016158,0.978110,0.984162,0.964899,0.948669,1.050057,...,0.988100,1.046720,1.055567,0.996797,0.994504,1.003505,1.028614,1.013192,1.001157,0.976282
2,0.940791,0.998123,1.035731,0.985416,0.991166,0.980330,1.000977,0.967662,1.001657,0.995238,...,0.974803,1.077138,1.053612,0.997355,1.036102,1.021395,1.001555,1.030414,0.966340,0.975684
3,0.953345,1.018694,1.024122,0.978357,0.983719,1.028451,0.992787,0.956969,0.974010,1.001192,...,0.949780,1.062309,1.020378,1.028535,1.057924,1.012154,0.978211,1.023243,0.960675,0.978174
4,0.927972,1.061737,0.996108,1.018492,1.012546,1.001936,1.012230,0.935077,0.970201,1.041614,...,0.946384,1.052002,1.050675,1.020409,1.125435,1.043800,0.986489,1.019860,0.942527,0.984915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.270530,3.511035,1.361440,1.440385,1.552888,1.813367,2.126600,1.699247,0.608688,1.748434,...,1.367967,0.672843,1.491698,0.986047,1.371893,1.432051,1.662495,1.867995,1.392601,1.803029
249,1.242557,3.377424,1.332814,1.398532,1.558273,1.812415,2.137144,1.727715,0.603242,1.743721,...,1.410501,0.668024,1.482587,0.946936,1.315029,1.423991,1.666313,1.919544,1.426457,1.793563
250,1.285698,3.310083,1.353826,1.423200,1.575124,1.841386,2.151946,1.701572,0.583932,1.729607,...,1.421518,0.645652,1.494081,0.899303,1.301959,1.354386,1.684934,1.941546,1.453297,1.764009
251,1.295477,3.396109,1.377769,1.383681,1.581831,1.740906,2.108383,1.656448,0.581859,1.691529,...,1.459160,0.636693,1.557859,0.910118,1.277947,1.345249,1.690017,1.940457,1.401684,1.759207


In [46]:
MC_btc_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.994671,1.009004,1.024370,1.018590,1.021279,1.005770,1.006885,0.996866,0.994823,0.965909,...,0.971311,0.985645,0.999139,1.020784,0.988934,0.943781,0.997093,1.015801,1.020906,1.001831
2,1.035934,1.037595,1.061610,1.028216,1.002214,1.000466,0.956587,1.044799,0.988329,0.966223,...,0.977264,0.989860,0.980238,1.017991,1.013114,0.964305,1.008759,1.043123,1.044573,0.995310
3,1.084429,1.051738,1.071349,1.009596,1.010680,0.974098,0.989761,1.057919,0.973200,0.933150,...,0.930478,1.032542,0.967563,0.997528,0.991016,0.969172,0.989717,1.028226,1.011236,1.006396
4,1.049701,1.062565,1.044904,1.013411,1.033279,0.979847,0.975889,1.117755,0.968664,0.945284,...,0.920157,1.066088,1.004323,0.996968,0.980629,1.002654,1.018403,1.068825,1.008456,1.018535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.943813,1.039302,0.454105,1.275839,1.038556,2.027360,0.926612,2.574052,1.884694,0.772140,...,2.604989,2.202792,1.564302,3.338684,1.094964,1.288870,1.473120,2.282528,2.764120,1.738835
249,1.971614,1.059947,0.445101,1.239098,1.015965,2.012405,0.967566,2.590363,1.908543,0.809178,...,2.582550,2.187999,1.581368,3.324004,1.109910,1.355568,1.498909,2.408735,2.821389,1.773380
250,2.033507,1.072939,0.455730,1.239952,1.004966,1.999685,0.951812,2.591222,1.900117,0.817387,...,2.626622,2.149014,1.567169,3.427225,1.096625,1.286273,1.526635,2.481771,2.796462,1.739669
251,2.069003,1.092381,0.428204,1.270514,0.993007,2.108817,0.960469,2.572414,1.922806,0.813064,...,2.523006,2.168481,1.530572,3.485084,1.109191,1.333143,1.524738,2.493303,2.882254,1.748095


In [47]:
MC_btc_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.981813,1.000330,1.022880,1.029343,1.013988,1.036105,0.978104,0.972342,1.003676,0.990169,...,0.956423,1.046325,1.026682,0.969281,1.021377,1.013970,1.000047,0.985596,0.989306,1.051634
2,0.986370,1.034574,1.021797,1.049234,0.993002,1.028165,1.007857,1.002745,1.018801,1.032946,...,1.013928,1.035354,1.046094,0.995633,0.993357,1.041181,1.005769,0.990814,1.020129,1.061841
3,0.992314,1.014068,1.039935,1.047683,0.997373,1.029572,1.074647,0.944184,1.012079,1.057756,...,1.003443,1.073231,1.011634,0.995947,0.953679,1.037083,1.043614,0.985102,1.034737,1.059316
4,0.987051,1.015887,1.051743,1.079034,1.031500,1.014037,1.086154,0.920540,1.000313,1.055302,...,1.004962,1.066189,1.010060,0.996176,0.951602,1.019365,1.058662,0.968783,1.064788,1.077867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.985174,1.271939,1.975808,1.132594,2.720805,0.943328,1.178952,1.072880,1.796666,1.148310,...,1.488604,0.672739,1.476670,2.161734,1.413836,3.473077,1.627730,1.020932,1.872352,1.422427
249,2.024897,1.321470,2.002071,1.080742,2.687190,0.920631,1.194931,1.059922,1.821641,1.105995,...,1.469946,0.679487,1.506689,2.210084,1.362437,3.583589,1.688739,1.038784,1.872817,1.434164
250,2.046706,1.340602,2.018788,1.093213,2.688248,0.894996,1.194683,1.105319,1.871539,1.098973,...,1.453681,0.669366,1.499056,2.169011,1.372723,3.579422,1.707954,1.031237,1.875005,1.385843
251,2.056850,1.342934,1.998221,1.100597,2.840620,0.949774,1.230496,1.125302,1.882120,1.078519,...,1.434736,0.670321,1.527223,2.184425,1.340491,3.640427,1.688946,1.049852,1.845666,1.341457


In [48]:
MC_btc_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.999533,0.989560,0.979739,1.025850,1.008884,1.016857,0.969439,1.035500,1.030091,0.980057,...,0.952206,1.034116,0.978515,1.034208,1.032199,1.056446,0.977983,0.965739,1.010461,1.006114
2,0.988124,1.005012,0.997666,1.026950,1.041289,1.070366,0.937216,0.997854,1.048777,0.994770,...,0.974722,1.038811,0.974050,1.039126,1.038287,1.061632,0.991128,0.985539,1.017043,0.984334
3,0.941309,1.035018,0.995295,1.012738,1.044617,1.035951,0.967111,1.033654,1.043937,1.013640,...,0.997057,1.054559,0.979969,1.046806,1.071583,1.094358,1.021498,0.999131,0.975279,0.989979
4,0.943918,1.045812,0.972802,1.022935,1.054136,1.028498,0.957570,1.029629,1.087292,0.991987,...,1.008116,1.012814,0.965279,1.110535,1.043226,1.138808,1.064862,0.999629,0.984002,0.992378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.689880,2.295107,1.776465,2.093961,1.417337,2.097387,1.422582,1.105335,2.813665,1.065110,...,1.718994,1.597349,0.511962,0.918255,1.321615,2.154855,1.209312,1.105731,2.397719,1.104733
249,1.702927,2.245014,1.715793,2.035628,1.407443,2.120248,1.437110,1.132182,2.817275,1.076845,...,1.746518,1.726116,0.501227,0.934322,1.337795,2.174056,1.240496,1.138175,2.479082,1.126439
250,1.660759,2.172453,1.668005,2.016329,1.444009,2.119951,1.430259,1.162591,2.881075,1.048807,...,1.829483,1.753063,0.497319,0.934189,1.344021,2.154594,1.200269,1.094196,2.565090,1.066380
251,1.642406,2.079582,1.707442,2.088941,1.358216,2.198326,1.424205,1.207354,2.997279,1.017216,...,1.771986,1.704209,0.496228,0.961303,1.358020,2.257236,1.226571,1.079531,2.566968,1.062802


In [49]:
MC_btc_spy.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.018057,0.986008,1.013228,1.014866,1.016931,0.975030,1.010254,1.029609,0.929939,0.943357,...,0.959366,1.004656,0.980805,0.991238,1.041742,0.973602,1.005074,1.035334,1.014865,0.995168
2,1.005849,0.967225,0.993128,1.096570,0.980094,0.981883,1.017252,1.046259,0.921144,0.942133,...,0.932238,0.983129,0.978932,0.963159,1.050607,0.912597,1.022884,1.028328,1.044140,1.032673
3,1.028278,0.965829,0.985820,1.085911,0.992354,0.974394,1.007160,1.007231,0.941289,0.929601,...,0.914940,1.001498,0.972940,0.927745,1.062155,0.918021,1.014053,1.074169,1.018587,1.016156
4,1.035568,0.994510,0.970364,1.095008,0.925487,0.971869,1.004491,1.016818,0.939055,0.926150,...,0.887940,0.987631,0.979550,0.919208,1.051382,0.919172,0.961847,1.117297,1.035069,1.021229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.051930,1.143152,1.006746,2.606931,0.863924,1.988488,1.822043,1.312514,1.367655,1.250348,...,0.610712,1.370369,0.695159,1.452571,1.272198,1.236345,0.777228,2.120896,1.393914,1.192991
249,1.052116,1.138869,1.017680,2.600495,0.881598,1.966812,1.732682,1.281248,1.388620,1.229502,...,0.589056,1.323751,0.702047,1.479307,1.284637,1.235526,0.781071,2.163841,1.374119,1.252302
250,1.016225,1.112562,1.029841,2.555299,0.935918,1.942507,1.743280,1.271332,1.386249,1.216936,...,0.583708,1.368466,0.683595,1.427674,1.222081,1.268313,0.815043,2.133233,1.335178,1.273352
251,0.991485,1.094097,1.036022,2.467072,0.907035,1.953284,1.672700,1.306399,1.373779,1.228909,...,0.566327,1.374594,0.672808,1.419570,1.199575,1.254323,0.820362,2.148505,1.403722,1.319740


In [50]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
#btc_line_plot = MC_btc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwp_line_plot = MC_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iws_line_plot = MC_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
mgc_line_plot = MC_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwc_line_plot = MC_iwc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwp_line_plot = MC_btc_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iws_line_plot = MC_btc_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_mgc_line_plot = MC_btc_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwc_line_plot = MC_btc_iwc.plot_simulation()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
#btc_distribution_plot = MC_btc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwp_distribution_plot = MC_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iws_distribution_plot = MC_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
mgc_distribution_plot = MC_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwc_distribution_plot = MC_iwc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwp_distribution_plot = MC_btc_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iws_distribution_plot = MC_btc_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_mgc_distribution_plot = MC_btc_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwc_distribution_plot = MC_btc_iwc.plot_distribution()

In [51]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
#btc_table = MC_btc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
#print(btc_table)

In [52]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwp_table = MC_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwp_table)

count           1000.000000
mean               1.084351
std                0.311127
min                0.407951
25%                0.875056
50%                1.036009
75%                1.246316
max                2.646097
95% CI Lower       0.609387
95% CI Upper       1.836731
Name: 252, dtype: float64


In [53]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iws_table = MC_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iws_table)

count           1000.000000
mean               1.106542
std                0.336499
min                0.341293
25%                0.857972
50%                1.069169
75%                1.306990
max                2.922660
95% CI Lower       0.596414
95% CI Upper       1.890115
Name: 252, dtype: float64


In [54]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
mgc_table = MC_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(mgc_table)

count           1000.000000
mean               1.111820
std                0.277585
min                0.504358
25%                0.910709
50%                1.067928
75%                1.278023
max                2.186546
95% CI Lower       0.636853
95% CI Upper       1.743561
Name: 252, dtype: float64


In [55]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwc_table = MC_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwc_table)

count           1000.000000
mean               1.156091
std                0.400485
min                0.398790
25%                0.879275
50%                1.100107
75%                1.347181
max                2.814598
95% CI Lower       0.546824
95% CI Upper       2.171116
Name: 252, dtype: float64


In [56]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
spy_table = MC_spy.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(spy_table)

count           1000.000000
mean               1.109562
std                0.275382
min                0.467123
25%                0.901162
50%                1.086130
75%                1.273825
max                2.376144
95% CI Lower       0.676037
95% CI Upper       1.686902
Name: 252, dtype: float64


In [57]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwp_table = MC_btc_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwp_table)

count           1000.000000
mean               1.594306
std                0.652997
min                0.354053
25%                1.131077
50%                1.478922
75%                1.900135
max                5.115989
95% CI Lower       0.674822
95% CI Upper       3.144082
Name: 252, dtype: float64


In [58]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iws_table = MC_btc_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iws_table)

count           1000.000000
mean               1.665693
std                0.721203
min                0.347245
25%                1.140319
50%                1.519342
75%                2.038121
max                8.014698
95% CI Lower       0.674246
95% CI Upper       3.415903
Name: 252, dtype: float64


In [59]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_mgc_table = MC_btc_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_mgc_table)

count           1000.000000
mean               1.644775
std                0.697826
min                0.460648
25%                1.144077
50%                1.512447
75%                1.961888
max                6.566493
95% CI Lower       0.714575
95% CI Upper       3.464680
Name: 252, dtype: float64


In [60]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwc_table = MC_btc_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwc_table)

count           1000.000000
mean               1.654947
std                0.654533
min                0.506192
25%                1.172573
50%                1.525968
75%                2.066704
max                4.786771
95% CI Lower       0.720087
95% CI Upper       3.110305
Name: 252, dtype: float64


In [61]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_spy_table = MC_btc_spy.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_spy_table)

count           1000.000000
mean               1.629882
std                0.657180
min                0.474551
25%                1.178165
50%                1.493699
75%                1.943497
max                4.945330
95% CI Lower       0.711601
95% CI Upper       3.251296
Name: 252, dtype: float64


In [62]:
# Print the current balance of the stock and bond portion of the members portfolio
investment = 100000

In [63]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
#ci_lower_cumulative_return = round(btc_table[8] * investment, 2)
#ci_upper_cumulative_return = round(btc_table[9] * investment, 2)

# Print the result of your calculations
#print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
 #     f" with a 100% BTC over the next six months will end within the range of"
#      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

#mean = round(btc_table[1] * investment, 2)
#print(mean)

In [64]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWP over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWP over the next six months will end within the range of $60938.69 and $183673.12.
108435.05


In [65]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWS over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWS over the next six months will end within the range of $59641.36 and $189011.47.
110654.15


In [66]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% MGC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% MGC over the next six months will end within the range of $63685.26 and $174356.09.
111182.02


In [67]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWC over the next six months will end within the range of $54682.44 and $217111.56.
115609.13


In [68]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(spy_table[8] * investment, 2)
ci_upper_cumulative_return = round(spy_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% SPY over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(spy_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% SPY over the next six months will end within the range of $67603.74 and $168690.16.
110956.24


In [69]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of $67482.17 and $314408.17.
159430.57


In [70]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of $67424.6 and $341590.27.
166569.32


In [71]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of $71457.46 and $346467.98.
164477.5


In [72]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of $72008.73 and $311030.46.
165494.74


In [73]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_spy_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_spy_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and SPY respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_spy_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and SPY respectively over the next six months will end within the range of $71160.1 and $325129.65.
162988.16
